# Filter Analytics: Historical Analysis

This notebook demonstrates historical filter metrics analysis using Parquet data.

**Features:**
- Load Parquet data with partitioning
- Visualize filter trends over time
- Compare filter effectiveness across sources
- Detect anomalies in filter behavior
- Export insights and recommendations

**Requirements:**
```bash
pip install pandas pyarrow duckdb matplotlib seaborn plotly
```

## 1. Setup and Data Loading

In [ ]:
# Imports
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta

# Configure visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Paths
PARQUET_PATH = Path("../data/warehouse/filter_history.parquet")

print(f"Notebook initialized at {datetime.now()}")
print(f"Parquet path: {PARQUET_PATH}")

In [ ]:
# Load Parquet data
print("Loading Parquet data...")
df = pd.read_parquet(PARQUET_PATH)

print(f"✅ Loaded {len(df):,} filter records")
print(f"   Date range: {df['date'].min()} to {df['date'].max()}")
print(f"   Sources: {df['source'].nunique()} unique")
print(f"   Filter types: {df['filter_type'].nunique()} unique")
print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Preview data
df.head(10)

In [ ]:
# Data types and summary
df.info()

## 2. Incremental vs Full Processing Demo

In [ ]:
# Show how incremental processing works
print("Incremental Processing Benefits:")
print("=" * 60)

# Count unique run_ids
unique_runs = df['run_id'].nunique()
total_records = len(df)

print(f"\n📊 Dataset Stats:")
print(f"   Unique runs: {unique_runs:,}")
print(f"   Total filter records: {total_records:,}")
print(f"   Avg records per run: {total_records / unique_runs:.1f}")

print(f"\n🚀 Incremental Mode Advantages:")
print(f"   - Skip {unique_runs:,} already-processed run_ids")
print(f"   - Process only new metrics files")
print(f"   - Merge results with existing Parquet")
print(f"   - No data duplication (dedupe on run_id + filter_type)")

print(f"\n💾 Storage Efficiency:")
parquet_size = sum(f.stat().st_size for f in PARQUET_PATH.rglob("*.parquet"))
print(f"   Parquet size: {parquet_size / 1024**2:.2f} MB")
print(f"   Partitions: source={df['source'].nunique()}, month={df['month'].nunique()}")

## 3. Visualization: Filter Trends Over Time

In [ ]:
# Time series: Filter counts over time by source
print("Creating time series visualization...")

# Aggregate by date and filter type
daily_trends = df.groupby([pd.Grouper(key='date', freq='D'), 'filter_label'])['records_filtered'].sum().reset_index()

# Get top 5 filters by total volume
top_filters = df.groupby('filter_label')['records_filtered'].sum().nlargest(5).index

plt.figure(figsize=(16, 8))

for filter_label in top_filters:
    subset = daily_trends[daily_trends['filter_label'] == filter_label]
    plt.plot(subset['date'], subset['records_filtered'], label=filter_label, marker='o', linewidth=2)

plt.xlabel('Date', fontsize=12)
plt.ylabel('Records Filtered', fontsize=12)
plt.title('Filter Trends Over Time (Top 5 Filters by Volume)', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"✅ Time series chart created for {len(top_filters)} top filters")

## 4. Visualization: Filter Rate Heatmap

In [ ]:
# Heatmap: Filter rates by source and filter type
print("Creating filter rate heatmap...")

# Calculate average filter rate by source and filter type
heatmap_data = df.groupby(['source', 'filter_label'])['filter_rate'].mean().unstack(fill_value=0)

# Convert to percentages
heatmap_data = heatmap_data * 100

plt.figure(figsize=(14, 8))
sns.heatmap(
    heatmap_data,
    annot=True,
    fmt='.2f',
    cmap='YlOrRd',
    cbar_kws={'label': 'Filter Rate (%)'},
    linewidths=0.5
)
plt.title('Average Filter Rates by Source and Filter Type (%)', fontsize=14, fontweight='bold')
plt.xlabel('Filter Type', fontsize=12)
plt.ylabel('Source', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(f"✅ Heatmap created for {len(heatmap_data)} sources x {len(heatmap_data.columns)} filters")

## 5. Visualization: Top Filters by Total Rejections

In [ ]:
# Bar chart: Top 10 filters by total rejections
print("Creating top filters bar chart...")

top_10_filters = df.groupby('filter_label')['records_filtered'].sum().nlargest(10).sort_values()

plt.figure(figsize=(12, 8))
top_10_filters.plot(kind='barh', color='steelblue')
plt.xlabel('Total Records Filtered', fontsize=12)
plt.ylabel('Filter Type', fontsize=12)
plt.title('Top 10 Filters by Total Rejections', fontsize=14, fontweight='bold')
plt.grid(True, axis='x', alpha=0.3)

# Add value labels
for i, v in enumerate(top_10_filters.values):
    plt.text(v, i, f' {v:,.0f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print(f"✅ Bar chart created showing top {len(top_10_filters)} filters")

## 6. DuckDB SQL Query Examples

In [ ]:
# Example DuckDB queries
print("Running DuckDB SQL queries...")

con = duckdb.connect()

# Query 1: Weekly aggregation
query1 = f"""
SELECT
    DATE_TRUNC('week', timestamp) AS week,
    source,
    SUM(records_filtered) AS total_filtered,
    COUNT(DISTINCT filter_type) AS filter_types_used
FROM read_parquet('{PARQUET_PATH}/**/*.parquet')
GROUP BY week, source
ORDER BY week DESC, total_filtered DESC
LIMIT 20
"""

result1 = con.execute(query1).df()
print("\n📊 Query 1: Weekly Filter Summary")
print(result1.to_string(index=False))

In [ ]:
# Query 2: Filter category breakdown
query2 = f"""
SELECT
    filter_category,
    COUNT(DISTINCT filter_type) AS filter_count,
    SUM(records_filtered) AS total_filtered,
    AVG(filter_rate) * 100 AS avg_rate_pct
FROM read_parquet('{PARQUET_PATH}/**/*.parquet')
GROUP BY filter_category
ORDER BY total_filtered DESC
"""

result2 = con.execute(query2).df()
print("\n📊 Query 2: Filter Category Breakdown")
print(result2.to_string(index=False))

In [ ]:
# Query 3: Source quality comparison
query3 = f"""
WITH SourceStats AS (
    SELECT DISTINCT
        run_id,
        source,
        records_received,
        records_passed
    FROM read_parquet('{PARQUET_PATH}/**/*.parquet')
)
SELECT
    source,
    COUNT(*) AS total_runs,
    SUM(records_received) AS total_received,
    SUM(records_passed) AS total_passed,
    AVG(records_passed::FLOAT / NULLIF(records_received, 0)) * 100 AS avg_pass_rate_pct
FROM SourceStats
GROUP BY source
ORDER BY total_received DESC
"""

result3 = con.execute(query3).df()
print("\n📊 Query 3: Source Quality Comparison")
print(result3.to_string(index=False))

## 7. Insights and Recommendations

In [ ]:
# Generate insights
print("Generating insights...\n")
print("=" * 80)
print("  KEY INSIGHTS FROM FILTER ANALYTICS")
print("=" * 80)

# Insight 1: Most impactful filter
top_filter = df.groupby('filter_label')['records_filtered'].sum().idxmax()
top_filter_count = df.groupby('filter_label')['records_filtered'].sum().max()
print(f"\n1️⃣ Most Impactful Filter:")
print(f"   {top_filter}")
print(f"   Total rejections: {top_filter_count:,}")

# Insight 2: Highest quality source
source_quality = df.groupby('source').apply(
    lambda x: (x['records_passed'].iloc[0] / x['records_received'].iloc[0]) * 100
).sort_values(ascending=False)
best_source = source_quality.idxmax()
best_source_rate = source_quality.max()
print(f"\n2️⃣ Highest Quality Source:")
print(f"   {best_source}")
print(f"   Average pass rate: {best_source_rate:.2f}%")

# Insight 3: Most active filter category
category_activity = df.groupby('filter_category')['records_filtered'].sum().sort_values(ascending=False)
top_category = category_activity.idxmax()
top_category_count = category_activity.max()
print(f"\n3️⃣ Most Active Filter Category:")
print(f"   {top_category}")
print(f"   Total rejections: {top_category_count:,}")

# Insight 4: Date range coverage
date_range = (df['date'].max() - df['date'].min()).days
print(f"\n4️⃣ Historical Coverage:")
print(f"   Date range: {df['date'].min()} to {df['date'].max()}")
print(f"   Total days: {date_range}")
print(f"   Data density: {len(df) / date_range:.1f} records/day")

# Insight 5: Filter diversity
filters_per_source = df.groupby('source')['filter_type'].nunique()
print(f"\n5️⃣ Filter Diversity by Source:")
for source, count in filters_per_source.items():
    print(f"   {source}: {count} unique filters")

print("\n" + "=" * 80)

In [ ]:
# Recommendations
print("\n📋 RECOMMENDATIONS")
print("=" * 80)

print("\n1. Filter Optimization:")
print("   - Review top 3 filters for potential relaxation")
print("   - Consider if rejection thresholds are too strict")
print("   - Monitor filter rate trends for sudden changes")

print("\n2. Source Quality:")
print("   - Investigate low-quality sources (high rejection rates)")
print("   - Share best practices from high-quality sources")
print("   - Consider source-specific filter tuning")

print("\n3. Data Collection:")
print("   - Continue incremental Parquet exports after each pipeline run")
print("   - Set up automated weekly trend reports")
print("   - Archive old metrics JSON to reduce storage")

print("\n4. Monitoring:")
print("   - Set up alerts for anomalous filter rates (>2σ deviation)")
print("   - Track filter effectiveness over time")
print("   - Review new filter types quarterly")

print("\n" + "=" * 80)

## 8. Export Summary Statistics

In [ ]:
# Export summary to CSV
output_dir = Path("../data/reports")
output_dir.mkdir(parents=True, exist_ok=True)

# Summary 1: Filter effectiveness
filter_summary = df.groupby(['filter_label', 'filter_category']).agg({
    'records_filtered': ['sum', 'mean', 'count'],
    'filter_rate': ['mean', 'std']
}).round(4)

filter_summary.columns = ['_'.join(col) for col in filter_summary.columns]
filter_summary = filter_summary.reset_index().sort_values('records_filtered_sum', ascending=False)

summary_file = output_dir / "filter_effectiveness_summary.csv"
filter_summary.to_csv(summary_file, index=False)
print(f"✅ Exported filter effectiveness summary to: {summary_file}")

# Summary 2: Source quality
source_summary = df.groupby('source').agg({
    'run_id': 'nunique',
    'records_received': 'sum',
    'records_passed': 'sum',
    'records_filtered': 'sum'
})

source_summary['pass_rate'] = (source_summary['records_passed'] / source_summary['records_received'] * 100).round(2)
source_summary = source_summary.reset_index().sort_values('records_received', ascending=False)

source_file = output_dir / "source_quality_summary.csv"
source_summary.to_csv(source_file, index=False)
print(f"✅ Exported source quality summary to: {source_file}")

print("\n📊 Summary exports complete!")

## Conclusion

This notebook demonstrated:

1. **Loading Parquet data** with Hive-style partitioning
2. **Incremental processing benefits** - skip already-processed run_ids
3. **Time series visualization** - filter trends over time
4. **Heatmap analysis** - filter rates by source and type
5. **Top filter identification** - highest rejection counts
6. **DuckDB SQL queries** - efficient analytical queries
7. **Insights generation** - actionable recommendations
8. **Data export** - summary statistics to CSV

**Next Steps:**
- Set up automated weekly exports
- Create dashboard with real-time filter metrics
- Implement anomaly detection alerts
- Optimize filter thresholds based on historical data